In [3]:
import numpy as np
import pandas as pd
import datetime as dt
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from lifelines import WeibullAFTFitter
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import scipy.special as sc
import math

In [ ]:
df = pd.read_csv("C:/Users/jvgat/Downloads/Hazard/df_after_feature_engineering.csv")
df['published_date'] = pd.to_datetime(df['published_date'], format='mixed', utc=True)

df

C:\Users\jvgat\AppData\Local\Temp\ipykernel_2980\3355016506.py:1: DtypeWarning: Columns (34,35,36,37,39,40,41,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/jvgat/Downloads/Hazard/df_after_feature_engineering.csv")


,cve_id,cve_year,published_date,base_score,nvd_present,jvn_present,eu_present,kev_present,source_list,sources,...,tfidf_90,tfidf_91,tfidf_92,tfidf_93,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99
0,CVE-2021-0001,2021,2021-06-09 20:15:08.180000+00:00,4.7,True,False,False,False,['NVD'],NVD,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,CVE-2021-0002,2021,2021-08-11 13:15:15.513000+00:00,7.1,True,False,False,False,['NVD'],NVD,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,CVE-2021-0003,2021,2021-08-11 13:15:15.563000+00:00,5.5,True,False,False,False,['NVD'],NVD,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,CVE-2021-0004,2021,2021-08-11 13:15:15.600000+00:00,4.4,True,False,False,False,['NVD'],NVD,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,CVE-2021-0005,2021,2021-08-11 13:15:15.637000+00:00,4.4,True,False,False,False,['NVD'],NVD,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149406,CVE-2025-9994,2025,2025-09-09 14:15:49.943000+00:00,9.8,True,False,True,False,"['NVD', 'EUVD']","NVD,EUVD",...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
149407,CVE-2025-9996,2025,2025-09-09 21:15:39.893000+00:00,5.8,True,False,True,False,"['NVD', 'EUVD']","NVD,EUVD",...,0.0,0.0,0.214654,0.0,0.0,0.102374,0.0,0.0,0.0,0.0
149408,CVE-2025-9997,2025,2025-09-09 22:15:34.833000+00:00,5.8,True,False,True,False,"['NVD', 'EUVD']","NVD,EUVD",...,0.0,0.0,0.000000,0.0,0.0,0.123009,0.0,0.0,0.0,0.0
149409,CVE-2025-9998,2025,2025-09-05 17:15:40.040000+00:00,6.0,True,False,True,False,"['NVD', 'EUVD']","NVD,EUVD",...,0.0,0.0,0.000000,0.0,0.0,0.241149,0.0,0.0,0.0,0.0


In [10]:
df['exploited'] = df['kev_present'].astype(int)
df['kev_punblished'] = pd.to_datetime(df['kev_published'], utc=True)
obs_end = pd.to_datetime('today', utc=True)
df['T'] = np.where(df['exploited']==1,
                           (df['kev_punblished'] - df['published_date']).dt.total_seconds()/(3600*24),
                           (obs_end - df['published_date']).dt.total_seconds()/(3600*24))
df['E'] = df['exploited'].astype(int)

df = df[df['T'] > 0].copy()

df


,cve_id,cve_year,published_date,base_score,nvd_present,jvn_present,eu_present,kev_present,source_list,sources,...,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99,exploited,kev_punblished,T,E
0,CVE-2021-0001,2021,2021-06-09 20:15:08.180000+00:00,4.7,True,False,False,False,['NVD'],NVD,...,0.0,0.000000,0.0,0.0,0.0,0.0,0,NaT,1615.554126,0
1,CVE-2021-0002,2021,2021-08-11 13:15:15.513000+00:00,7.1,True,False,False,False,['NVD'],NVD,...,0.0,0.000000,0.0,0.0,0.0,0.0,0,NaT,1552.845708,0
2,CVE-2021-0003,2021,2021-08-11 13:15:15.563000+00:00,5.5,True,False,False,False,['NVD'],NVD,...,0.0,0.000000,0.0,0.0,0.0,0.0,0,NaT,1552.845707,0
3,CVE-2021-0004,2021,2021-08-11 13:15:15.600000+00:00,4.4,True,False,False,False,['NVD'],NVD,...,0.0,0.000000,0.0,0.0,0.0,0.0,0,NaT,1552.845707,0
4,CVE-2021-0005,2021,2021-08-11 13:15:15.637000+00:00,4.4,True,False,False,False,['NVD'],NVD,...,0.0,0.000000,0.0,0.0,0.0,0.0,0,NaT,1552.845706,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149406,CVE-2025-9994,2025,2025-09-09 14:15:49.943000+00:00,9.8,True,False,True,False,"['NVD', 'EUVD']","NVD,EUVD",...,0.0,0.000000,0.0,0.0,0.0,0.0,0,NaT,62.803642,0
149407,CVE-2025-9996,2025,2025-09-09 21:15:39.893000+00:00,5.8,True,False,True,False,"['NVD', 'EUVD']","NVD,EUVD",...,0.0,0.102374,0.0,0.0,0.0,0.0,0,NaT,62.512092,0
149408,CVE-2025-9997,2025,2025-09-09 22:15:34.833000+00:00,5.8,True,False,True,False,"['NVD', 'EUVD']","NVD,EUVD",...,0.0,0.123009,0.0,0.0,0.0,0.0,0,NaT,62.470484,0
149409,CVE-2025-9998,2025,2025-09-05 17:15:40.040000+00:00,6.0,True,False,True,False,"['NVD', 'EUVD']","NVD,EUVD",...,0.0,0.241149,0.0,0.0,0.0,0.0,0,NaT,66.678757,0


In [11]:
# --------------------------
# 2) Choose features for static (generative) model
# --------------------------
# numeric features
numeric_features = ['base_score']
# boolean flags (cast to int)
flag_cols = [c for c in ['nvd_present','jvn_present','eu_present','kev_present'] if c in df.columns]
# TF-IDF columns (detect automatically by prefix)
tfidf_cols = [c for c in df.columns if c.startswith('tfidf_')]

# final static feature list (you can expand with CWE, vendor popularity, etc.)
static_features = numeric_features + flag_cols + tfidf_cols

# ensure no NaNs
X = df[static_features].fillna(0).copy()

# Standardize numeric before GaussianNB? GaussianNB works without scaling but scaling helps if magnitudes differ.
scaler = StandardScaler(with_mean=True, with_std=True)
X_scaled = X.copy()
if len(tfidf_cols) > 0:
    # scale only numerical columns (including TFIDF)
    X_scaled[static_features] = scaler.fit_transform(X[static_features])

In [13]:
# --------------------------
# 3) Fit Gaussian Naive Bayes to approximate p(x|y)
# --------------------------
y = df['exploited'].astype(int).values
gnb = GaussianNB()
gnb.fit(X_scaled.values, y)

# Helper: compute per-class log-likelihood for each sample (Gaussian NB analytic form)
def gnb_log_likelihood_matrix(X_arr, gnb_model):
    # returns array shape (n_samples, n_classes) of log p(x | class)
    # using gnb.theta_ (means) and gnb.var_ (variances) - CHANGED from sigma_
    theta = gnb_model.theta_       # shape (n_classes, n_features)
    var = gnb_model.var_           # shape (n_classes, n_features) - CHANGED
    n_samples = X_arr.shape[0]
    n_classes = theta.shape[0]
    ll = np.zeros((n_samples, n_classes))
    # gaussian log-prob: -0.5 * sum(log(2*pi*var) + (x - mu)^2 / var)
    const = -0.5 * np.log(2 * math.pi)
    for k in range(n_classes):
        var_k = var[k] + 1e-12
        ll[:, k] = np.sum(const - 0.5 * np.log(var_k) - 0.5 * ((X_arr - theta[k])**2) / var_k, axis=1)
    return ll

X_arr = X_scaled.values
ll_matrix = gnb_log_likelihood_matrix(X_arr, gnb)
df['ll_non'] = ll_matrix[:, 0]
df['ll_exploit'] = ll_matrix[:, 1]
df['llr_static'] = df['ll_exploit'] - df['ll_non']

# Per-feature contributions: log p(x_j | y=1) - log p(x_j | y=0) (useful for explanations)
# compute per-feature contribution arrays for top-k inspection
def per_feature_llr(x_row, gnb_model):
    # returns vector length n_features
    theta = gnb_model.theta_
    var = gnb_model.var_  # CHANGED from sigma_
    # compute per-feature log-prob for each class
    # feature wise: lp_kj = -0.5*(log(2pi*var_kj) + (xj-mu_kj)^2/var_kj)
    x = x_row.values
    lp_k = np.zeros((2, len(x)))
    for k in [0,1]:
        var_k = var[k] + 1e-12
        lp_k[k,:] = -0.5*(np.log(2*np.pi*var_k) + ((x - theta[k])**2)/var_k)
    return lp_k[1,:] - lp_k[0,:]

In [21]:
# --------------------------
# 4) Survival model: Weibull AFT (time-to-exploit)
# --------------------------
from lifelines import WeibullAFTFitter
from sklearn.decomposition import TruncatedSVD
import numpy as np
import pandas as pd
import math

# --- optional: reduce high-dimensional TF-IDF features ---
tfidf_cols = [c for c in df.columns if c.startswith("tfidf_")]
if len(tfidf_cols) > 30:
    print(f"Reducing {len(tfidf_cols)} TF-IDF columns → 20 SVD components")
    svd = TruncatedSVD(n_components=20, random_state=42)
    tfidf_reduced = svd.fit_transform(df[tfidf_cols].fillna(0))
    tfidf_reduced_df = pd.DataFrame(tfidf_reduced,
                                    columns=[f"tfidf_svd{i}" for i in range(20)],
                                    index=df.index)
    df = pd.concat([df.drop(columns=tfidf_cols), tfidf_reduced_df], axis=1)
    tfidf_cols = list(tfidf_reduced_df.columns)

# --- select covariates ---
aft_cols = [c for c in ['base_score','nvd_present','jvn_present','eu_present','kev_present'] if c in df.columns]
aft_cols += tfidf_cols

aft_df = df[[*aft_cols, 'T', 'E']].copy()

# --- convert bools to ints and ensure numeric types ---
for c in aft_cols:
    if aft_df[c].dtype == bool:
        aft_df[c] = aft_df[c].astype(int)
aft_df[aft_cols] = aft_df[aft_cols].apply(pd.to_numeric, errors='coerce')

# --- handle missing / infinite values ---
aft_df = aft_df.replace([np.inf, -np.inf], np.nan)
aft_df[aft_cols] = aft_df[aft_cols].fillna(0)
aft_df['T'] = aft_df['T'].replace([np.inf, -np.inf], np.nan).fillna(aft_df['T'].median())
aft_df['E'] = aft_df['E'].fillna(0).astype(int)

# --- scale durations for numeric stability ---
aft_df['T'] = aft_df['T'] / 100.0  # convert days → "hundred-days" units

# --- remove columns with near-zero variance (can cause complete separation) ---
low_var = aft_df[aft_cols].nunique() <= 1
drop_cols = list(low_var[low_var].index)
if drop_cols:
    print("Dropping low-variance columns:", drop_cols)
    aft_cols = [c for c in aft_cols if c not in drop_cols]
    aft_df = aft_df[[*aft_cols, 'T', 'E']]

# --- fit Weibull AFT with penalizer & robust optimizer ---
weib = WeibullAFTFitter(penalizer=0.05)
weib._scipy_fit_method = "SLSQP"

print(f"Fitting Weibull AFT on {len(aft_df)} CVEs, {len(aft_cols)} covariates...")
weib.fit(aft_df, duration_col='T', event_col='E', show_progress=True)

print("\n--- Weibull AFT Summary ---")
display(weib.summary)

# --- helper to compute log f(t) and log S(t) ---
def weib_logf_logs(row_features, t, model, delta=1e-3):
    """Return (log f(t), log S(t)) for one sample."""
    row = pd.DataFrame([row_features])
    S_t = model.predict_survival_function(row, times=[t]).iloc[:, 0].values[0]
    t2 = max(t + delta, t * 1.0001 + delta)
    S_t2 = model.predict_survival_function(row, times=[t2]).iloc[:, 0].values[0]
    f_t = max((S_t - S_t2) / (t2 - t), 1e-300)
    return math.log(f_t), math.log(max(S_t, 1e-300))

# --- compute Weibull logf/logS per CVE ---
weib_logf_list, weib_logs_list = [], []
for idx, r in df.iterrows():
    feat = {k: r[k] for k in aft_cols}
    t = max(r['T'] / 100.0, 1e-3)  # ensure positive
    lf, ls = weib_logf_logs(feat, t, weib)
    weib_logf_list.append(lf)
    weib_logs_list.append(ls)

df['weib_logf'] = weib_logf_list
df['weib_logs'] = weib_logs_list
print("Computed Weibull log-density (weib_logf) and log-survival (weib_logs).")



Fitting Weibull AFT on 148968 CVEs, 25 covariates...
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.012955674588719546
            Iterations: 40
            Function evaluations: 43
            Gradient evaluations: 40
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.012955674588719546
       x: [-2.374e-02 -3.844e-02 ...  8.313e+00  6.776e-02]
     nit: 40
     jac: [-6.610e-07  3.527e-07 ... -2.011e-07 -1.504e-06]
    nfev: 43
    njev: 40

--- Weibull AFT Summary ---


coef    exp(coef)  se(coef)  coef lower 95%  \
param   covariate                                                      
lambda_ base_score  -0.010607     0.989449  0.005051       -0.020507   
        eu_present  -0.077334     0.925581  0.022957       -0.122329   
        jvn_present  0.001351     1.001352  1.824420       -3.574446   
        kev_present -7.101227     0.000824  0.087715       -7.273145   
        nvd_present -0.000974     0.999026  0.156145       -0.307013   
        tfidf_svd0  -0.165818     0.847201  0.062809       -0.288922   
        tfidf_svd1  -0.083362     0.920018  0.071197       -0.222906   
        tfidf_svd10  0.033162     1.033718  0.115979       -0.194152   
        tfidf_svd11  0.052494     1.053896  0.119109       -0.180955   
        tfidf_svd12  0.214685     1.239471  0.120405       -0.021305   
        tfidf_svd13  0.092508     1.096922  0.125941       -0.154332   
        tfidf_svd14 -0.083275     0.920098  0.128316       -0.334771   
        tfidf_svd15 -0.096110     0.908364  0.131663       -0.354164   
        tfidf_svd16  0.099565     1.104691  0.130490       -0.156191   
        tfidf_svd17 -0.222573     0.800456  0.137185       -0.491452   
        tfidf_svd18  0.197438     1.218277  0.140157       -0.077265   
        tfidf_svd19  0.217199     1.242591  0.142162       -0.061433   
        tfidf_svd2   0.168505     1.183534  0.077538        0.016533   
        tfidf_svd3   0.061352     1.063273  0.085343       -0.105916   
        tfidf_svd4   0.123928     1.131934  0.091228       -0.054876   
        tfidf_svd5  -0.096320     0.908173  0.093766       -0.280099   
        tfidf_svd6  -0.005772     0.994245  0.107344       -0.216162   
        tfidf_svd7  -0.006473     0.993548  0.105969       -0.214169   
        tfidf_svd8  -0.032582     0.967943  0.111089       -0.250313   
        tfidf_svd9   0.018936     1.019116  0.113995       -0.204490   
        Intercept    8.313350  4077.949605  0.177826        7.964817   
rho_    Intercept    0.067761     1.070110  0.009565        0.049015   

                     coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%  \
param   covariate                                                               
lambda_ base_score        -0.000707             0.979702             0.999294   
        eu_present        -0.032338             0.884857             0.968179   
        jvn_present        3.577148             0.028031            35.771361   
        kev_present       -6.929309             0.000694             0.000979   
        nvd_present        0.305064             0.735641             1.356712   
        tfidf_svd0        -0.042714             0.749071             0.958185   
        tfidf_svd1         0.056182             0.800190             1.057790   
        tfidf_svd10        0.260477             0.823533             1.297548   
        tfidf_svd11        0.285943             0.834473             1.331017   
        tfidf_svd12        0.450674             0.978920             1.569370   
        tfidf_svd13        0.339347             0.856987             1.404031   
        tfidf_svd14        0.168220             0.715502             1.183197   
        tfidf_svd15        0.161945             0.701760             1.175795   
        tfidf_svd16        0.355321             0.855396             1.426639   
        tfidf_svd17        0.046305             0.611738             1.047394   
        tfidf_svd18        0.472140             0.925645             1.603422   
        tfidf_svd19        0.495830             0.940416             1.641861   
        tfidf_svd2         0.320477             1.016670             1.377785   
        tfidf_svd3         0.228621             0.899500             1.256865   
        tfidf_svd4         0.302731             0.946603             1.353551   
        tfidf_svd5         0.087459             0.755709             1.091397   
        tfidf_svd6         0.204619             0.805605             1.22

Computed Weibull log-density (weib_logf) and log-survival (weib_logs).


In [22]:
# --------------------------
# 5) Combine scores
# --------------------------
# standardize components to make scales comparable
df['llr_static_z'] = (df['llr_static'] - df['llr_static'].mean()) / (df['llr_static'].std() + 1e-12)
df['weib_logf_z'] = (df['weib_logf'] - df['weib_logf'].mean()) / (df['weib_logf'].std() + 1e-12)

# weights (tune via CV)
w1, w2 = 1.0, 1.0
df['combined'] = w1 * df['llr_static_z'] + w2 * df['weib_logf_z']

# map combined -> probability using logistic (Platt-like). Optionally calibrate on validation set.
df['prob'] = sc.expit(df['combined'])
# scale to 0-10 like CVSS
df['risk_0_10'] = 10 * df['prob']

In [24]:
# --------------------------
# 6) Evaluation (if you have labels)
# --------------------------
from sklearn.metrics import roc_auc_score, precision_score

# Drop rows with NaN in either exploited or prob before evaluating
df_eval = df.copy()
df_eval = df_eval.dropna(subset=['exploited', 'prob'])

# Ensure exploited is binary int
df_eval['exploited'] = df_eval['exploited'].astype(int)

# Check that we have both classes present
if df_eval['exploited'].nunique() > 1:
    # Clean any infs (just in case)
    df_eval = df_eval.replace([np.inf, -np.inf], np.nan).dropna(subset=['prob'])

    auc_score = roc_auc_score(df_eval['exploited'], df_eval['prob'])
    print(f"AUC (combined): {auc_score:.4f}")

    # Precision@K (top 100)
    topk = 100
    topk_idx = df_eval.sort_values('prob', ascending=False).head(topk).index
    prec_at_k = df_eval.loc[topk_idx, 'exploited'].mean()
    print(f"Precision@{topk}: {prec_at_k:.4f}")

    # Optional: survival concordance (time-aware validation)
    try:
        from lifelines.utils import concordance_index
        c_index = concordance_index(df_eval['T'], -df_eval['weib_logf'], df_eval['E'])
        print(f"Weibull concordance index (c-index): {c_index:.4f}")
    except Exception as e:
        print("Could not compute concordance:", e)
else:
    print("AUC cannot be computed — only one class present in exploited labels.")


AUC (combined): 1.0000
Precision@100: 1.0000
Weibull concordance index (c-index): 0.9996


In [25]:
# --------------------------
# 7) Explainability: top contributing features for a sample
# --------------------------
def top_k_feature_llr(idx, k=10):
    row = X_scaled.iloc[idx]
    contributions = per_feature_llr(row, gnb)
    feats = X_scaled.columns.tolist()
    pairs = sorted(zip(feats, contributions), key=lambda x: x[1], reverse=True)
    return pairs[:k], pairs[-k:]

# example for first CVE
top_pos, top_neg = top_k_feature_llr(df.index[0], k=8)
print("Top positive contributions to LLR:", top_pos)
print("Top negative contributions to LLR:", top_neg)

Top positive contributions to LLR: [('tfidf_98', 10.381075155894024), ('nvd_present', 10.365881682211944), ('tfidf_1', 10.364869043548353), ('jvn_present', 10.36312068260463), ('tfidf_67', 1.2463312655853431), ('tfidf_54', 1.1498340329809773), ('tfidf_27', 0.99177696050902), ('tfidf_31', 0.9163829588612813)]
Top negative contributions to LLR: [('tfidf_25', -0.8696955560240627), ('tfidf_35', -0.8751529703453678), ('tfidf_70', -0.9230534367033824), ('tfidf_74', -1.0059514604866466), ('tfidf_86', -1.0525814256963621), ('base_score', -2.3108514861825338), ('eu_present', -289.1130775417468), ('kev_present', -127697350822.3295)]


In [26]:
df_out = df[['cve_id','published_date','T','E','exploited','llr_static','weib_logf','combined','prob','risk_0_10']]

df_out

,cve_id,published_date,T,E,exploited,llr_static,weib_logf,combined,prob,risk_0_10
0,CVE-2021-0001,2021-06-09 20:15:08.180000+00:00,1615.554126,0,0,-1.276974e+11,-8.581815,-0.071716,0.482079,4.820788
1,CVE-2021-0002,2021-08-11 13:15:15.513000+00:00,1552.845708,0,0,-1.276974e+11,-8.557307,-0.017583,0.495604,4.956043
2,CVE-2021-0003,2021-08-11 13:15:15.563000+00:00,1552.845707,0,0,-1.276974e+11,-8.575417,-0.057585,0.485608,4.856078
3,CVE-2021-0004,2021-08-11 13:15:15.600000+00:00,1552.845707,0,0,-1.276974e+11,-8.587869,-0.085087,0.478741,4.787411
4,CVE-2021-0005,2021-08-11 13:15:15.637000+00:00,1552.845706,0,0,-1.276974e+11,-8.587869,-0.085087,0.478741,4.787411
...,...,...,...,...,...,...,...,...,...,...
149406,CVE-2025-9994,2025-09-09 14:15:49.943000+00:00,62.803642,0,0,-1.276974e+11,-8.670541,-0.267694,0.433473,4.334734
149407,CVE-2025-9996,2025-09-09 21:15:39.893000+00:00,62.512092,0,0,-1.276974e+11,-8.637628,-0.194996,0.451405,4.514048
149408,CVE-2025-9997,2025-09-09 22:15:34.833000+00:00,62.470484,0,0,-1.276974e+11,-8.664998,-0.255450,0.436482,4.364824
149409,CVE-2025-9998,2025-09-05 17:15:40.040000+00:00,66.678757,0,0,-1.276974e+11,-8.577297,-0.061736,0.484571,4.845709
